In [271]:
import pandas as pd
import json

#file_name = 'shiyanzu'
file_name = 'duizhaozu'
path = '/content/sample_data/' + file_name + '.xlsx'
shiyan_df = pd.read_excel(path, index_col=0)
#print(shiyan_df.head())

names = shiyan_df['Student Name 1']
student_name = []
for name in names:
    if name not in student_name:
        student_name.append(name)

**表格-1.6：**

1.6.1: 表格填写乱码（数字或英文）-> index

1.6.2: 最终的作答+表格只有一行 -> index2

In [272]:
#####################
### Section 1.6.1 ###
#####################

df_1_6_1 =  shiyan_df.loc[shiyan_df['Step Title'].str.contains('1.6') & ~shiyan_df['Teacher Comment'].isnull()]
answer = ['铝', '·铝', '木头', '泡沫塑料', '普通塑料', '粘土', '玻璃', '塑料', '黏土', '热', '暖', '冷', '高温', '中温', '低温', '5', '40', '50', '90']
index = []
flag1 = 0
for name in student_name:
    flag1 = 0
    df = df_1_6_1.loc[df_1_6_1['Student Name 1'] == name][:1]
    if len(df) == 0:
        continue
    #index.append(int(df.index)    
    json_df = df['Student Data']
    for data in json_df:
        json_obj = json.loads(data)['tableData']
        for row_data in json_obj:
            for cell in row_data:
                if cell['editable'] == True:
                    if len(cell['text']) > 0 and cell['text'] not in answer:
                        #print(cell['text'])
                        flag1 = 1
        #if flag1 == 1:
            #print(json_obj)
    if flag1:
        #index.append((name, df.index.to_list()[0], flag1))
        index.append((name, flag1))


#####################
### Section 1.6.2 ###
#####################

df_1_6_2 =  shiyan_df.loc[shiyan_df['Step Title'].str.contains('1.6') & ~shiyan_df['Teacher Score'].isnull()]
#print(df_1_6_2)
index2 = []
flag2 = 0
flag3 = 0
for name in student_name:
    flag2 = 0
    flag3 = 0
    df = df_1_6_2.loc[df_1_6_2['Student Name 1'] == name][-1:]
    if len(df) == 0:
        continue
    json_df = df['Student Data']
    for data in json_df:
        json_obj = json.loads(data)['tableData']
        row_num = 0 
        count = 0
        for row_data in json_obj:
            if row_num > 0:
                for cell in row_data:
                    # 第一行和第二行答案
                    if row_num == 1 or row_num == 2:
                        if cell['editable'] == True and len(cell['text']) > 0 and cell['text'] in answer:
                            count += 1
                        elif cell['editable'] == True and cell['text'] not in answer:
                            flag2 = 1
                    elif cell['editable'] == True and cell['text'] not in answer:
                        flag2 = 1
            row_num += 1
        if count != 6:
            flag3 = 2
    if flag2 or flag3 == 2:
        #index2.append((name, df.index.to_list()[0], 1))
        index2.append((name, 1))

In [273]:
##################
### Result 1.6 ###
##################

print('### RESULT 1.6.1 ###')
print(index)
print('\n### RESULT 1.6.2 ###')
print(index2)

### RESULT 1.6.1 ###
[('qiyujie d', 1), ('sunhaoran d', 1), ('hanzongqiao d', 1), ('zhangxinyue d', 1)]

### RESULT 1.6.2 ###
[('lilingqi d', 1), ('wangdishu d', 1), ('fanjiale d', 1), ('zhaomengchun d', 1), ('wangyaoting d', 1), ('qiyujie d', 1), ('sunhaoran d', 1), ('hanzongqiao d', 1), ('zhangbohan d', 1), ('guoyiming d', 1), ('shakewei d', 1), ('dukangxu d', 1), ('wangzhenghan d', 1), ('liran d', 1), ('lizhiyao d', 1)]


**实施-1.7**

1.7.1: 尝试了多少种材料（0-6）-> material_tries

1.7.2: 情景和任务是否一致：所有记录的初始温度都是低温（5），属于不一致，其他为一致 -> init_temp_counter

1.7.3: 在实施部分花费的时间（如果有跨天的，分开天算）-> duration_list


In [274]:
#######################
### Section 1.7.1~2 ###
#######################

df_1_7_12 =  shiyan_df.loc[shiyan_df['Step Title'].str.contains('1.7') & shiyan_df['Component Type'].str.contains('Embedded')]
#print(df_1_7.head())
material_tries = []
init_temp_counter = []
for name in student_name:
    df = df_1_7_12.loc[df_1_7_12['Student Name 1'] == name]
    if (len(df) == 0):
        continue
    json_df = df['Student Data']
    material_seen = []
    counter = 0
    for data in json_df:
        json_obj = json.loads(data)
        # Checking num of material used
        if json_obj['material0'] not in material_seen:
            material_seen.append(json_obj['material0'])
        plot_data = json_obj['trial']['series'][0]
        # Checking initial temp
        if len(plot_data['data']) > 0:
            if plot_data['data'][0]['y'] < 10:
                counter += 1
    # 1.7.1 result
    material_tries.append((name, len(material_seen)))
    # 1.7.2 result
    flag4 = counter == len(df.index)
    if flag4:
        init_temp_counter.append((name, 1))


#####################
### Section 1.7.3 ###
#####################

from datetime import datetime
FMT = '%H:%M:%S'
df_1_7_3 = shiyan_df.loc[shiyan_df['Step Title'].str.contains('1.7')]
#print(df_1_7_3.head())
duration_list = []
for name in student_name:
    df = df_1_7_3.loc[df_1_7_3['Student Name 1'] == name]['Server Timestamp']
    if len(df) == 0:
        continue
    time_list = df.to_list()
    day_list = []
    duration = {}
    for i in range(0, len(time_list)):
        time_slice = time_list[i].split(' ')
        day = time_slice[2]
        time = time_slice[-1]
        if day not in day_list:
            duration[day] = [time]
            day_list.append(day)
        else:
            duration[day].append(time)
    total_duration = 0
    for day, time in duration.items():
        tdelta = datetime.strptime(time[-1][2:], FMT) - datetime.strptime(time[0][2:], FMT)
        duration[day] = tdelta.seconds
        total_duration += tdelta.seconds
    #duration_list.append((name, duration))
    duration_list.append((name, total_duration))

In [275]:
##################
### Result 1.7 ###
##################

print('### RESULT 1.7.1 ###')
print(material_tries)
print('\n### RESULT 1.7.2 ###')
print(init_temp_counter)
print('\n### RESULT 1.7.3 ###')
print(duration_list)

### RESULT 1.7.1 ###
[('lilingqi d', 4), ('litaiheng d', 6), ('guoyilin d', 2), ('wangdishu d', 1), ('gengyue d', 1), ('qinluyao d', 6), ('xujiayue d', 5), ('renyuxuan d', 5), ('baiyu d', 3), ('fanjiale d', 1), ('zhaomengchun d', 6), ('zhangyining d', 6), ('zhaorijieya d', 3), ('liuzixi d', 1), ('wuyixuan d', 6), ('caosiyuan d', 6), ('guanyang d', 6), ('lixinran d', 6), ('litianle d', 1), ('sunqianrao d', 6), ('duolan d', 6), ('guoyaning d', 2), ('gaoxingyu d', 1), ('wangxumin d', 4), ('liuxinyi d', 6), ('chenmeiyan d', 6), ('zhangjiqing d', 2), ('wangqianhao d', 6), ('shikaihan d', 3), ('wangmengke d', 6), ('wangyaoting d', 2), ('gongyanbo d', 6), ('zhangshuhe d', 6), ('liujiahao d', 4), ('guochenxiao d', 4), ('guanyizeng d', 6), ('fengchanghao d', 5), ('liuchenshuo d', 6), ('liupuyuan d', 5), ('zhaoyijin d', 4), ('chenghongjie d', 6), ('gaofangze d', 1), ('xuzifa d', 6), ('yujingyang d', 4), ('sunhaoran d', 2), ('zhiyuhao d', 6), ('zhaoyao d', 6), ('hanzongqiao d', 2), ('hanyuhang d'

**结果-1.8**

1.8.1: 1.8的开放题是否有反馈，是否修改 -> modification_list

In [276]:
#####################
### Section 1.8.1 ###
#####################

df_1_8 = shiyan_df.loc[shiyan_df['Step Title'].str.contains('1.8')]
df_1_8_comment = df_1_8.loc[~shiyan_df['Teacher Comment'].isnull()]
#print(df_1_8.head())
modification_list = []
for name in student_name:
    flag5 = 0
    first_response = df_1_8_comment.loc[df_1_8_comment['Student Name 1'] == name][:1]['Response']
    if len(first_response) == 0:
        continue
    last_submit = df_1_8.loc[df_1_8['Student Name 1'] == name][-1:]['Response']
    if len(last_submit) == 0:
        continue
    if first_response.index == last_submit.index:
        score = df_1_8.loc[df_1_8['Student Name 1'] == name][-1:]['Teacher Score'].to_list()[0]
        if score < 4.0:
            flag5 = 1
    else:
        if len(first_response.to_list()) > 0:
            str1 = first_response.to_list()[0]
        else:
            str1 = ' '
        if len(last_submit.to_list()) > 0:
            str2 = last_submit.to_list()[0]
        else:
            str2 = ' '
        if str1 == str2:
            flag5 = 1
    if flag5 == 1:
        modification_list.append((name, flag5))

In [277]:
##################
### Result 1.8 ###
##################

print('### RESULT 1.8.1 ###')
print(modification_list)

### RESULT 1.8.1 ###
[('lilingqi d', 1), ('wangdishu d', 1), ('gengyue d', 1), ('xujiayue d', 1), ('zhaomengchun d', 1), ('wuyixuan d', 1), ('caosiyuan d', 1), ('sunqianrao d', 1), ('zhangjiqing d', 1), ('gongyanbo d', 1), ('qiyujie d', 1), ('zhangshuhe d', 1), ('liujiahao d', 1), ('guanyizeng d', 1), ('liupuyuan d', 1), ('zhaoyijin d', 1), ('gaofangze d', 1), ('zhiyuhao d', 1), ('zhaoyao d', 1), ('hanzongqiao d', 1), ('hanyuhang d', 1), ('guokaiyi d', 1), ('liuchang d', 1), ('wangyu d', 1), ('liugaojie d', 1), ('zhanghanyu d', 1), ('xiaboyang d', 1), ('zhangbohan d', 1), ('zhantianyi d', 1), ('guoyiming d', 1), ('zhangjinze d', 1), ('yangkairui d', 1), ('wuyinuo d', 1), ('lixingyi d', 1), ('wanghaobin d', 1), ('zhangzhenhao d', 1), ('wangzhenghan d', 1), ('mengyifu d', 1), ('kuangzexian d', 1), ('caolan d', 1), ('wuwei d', 1), ('jihao d', 1), ('zhousiyu d', 1), ('wuxinyue d', 1), ('hanzhujun d', 1), ('haoyihan d', 1)]


**结合数据**



In [278]:
"""
########################
### Print All Result ###
########################

print('### RESULT 1.6.1 ###')
print(index)
print('\n### RESULT 1.6.2 ###')
print(index2)
print('\n### RESULT 1.7.1 ###')
print(material_tries)
print('\n### RESULT 1.7.2 ###')
print(init_temp_counter)
print('\n### RESULT 1.7.3 ###')
print(duration_list)
print('\n### RESULT 1.8.1 ###')
print(modification_list)
print('\n')
"""


#####################
### Combine to DF ###
#####################

def set_val(array, result, name, binary=True):
    flag = 0
    for (k, v) in array:
        if k == name:
            if binary:
                flag = 1
            else:
                flag = v
    result.append(flag)

r_161 = []
r_162 = []
r_171 = []
r_172 = []
r_173 = []
r_181 = []
for name in student_name:
    set_val(index, r_161, name, True)
    set_val(index2, r_162, name, True)
    set_val(material_tries, r_171, name, False)
    set_val(init_temp_counter, r_172, name, True)
    set_val(duration_list, r_173, name, False)
    set_val(modification_list, r_181, name, True)
df_result = pd.DataFrame({
    'Student Name 1' : student_name,
    '表格填写乱码': r_161,
    '最终作答小于等于1行': r_162,
    '尝试材料数': r_171,
    '初始温度均为低温': r_172,
    '1.7花费时间': r_173,
    '反馈后无修改': r_181
})


######################
### Write to Excel ###
######################
output_file_name = file_name + '_output.xlsx'
writer = pd.ExcelWriter(output_file_name)
df_result.to_excel(writer)
writer.save()
print('==> Write Excel to path:', output_file_name)

==> Write Excel to path: duizhaozu_output.xlsx
